# Linear regressions on spiracle data with non-parametric confidence intervals for slopes

To begin with, we need to import necessary python packages.

In [1]:
import numpy as np
import pandas as pd

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

Loading BokehJS ...

We can now read in the data into a dataframe for analyis.

In [3]:
df = pd.read_csv("./20190322_supp_table_2.csv")

We take a look at the format for the data.

In [4]:
df.head()

,subfamily,species,sex,mass (g),spiracle,area (mm^2),depth (mm)
0,Cetoniinae,Goliathus goliathus,M,16.280,6,0.274408,2.512648
1,Cetoniinae,Goliathus goliathus,F,18.150,6,0.134949,1.606189
2,Cetoniinae,Coelorrhina hornimani,M,1.130,6,0.212131,0.553833
3,Cetoniinae,Dicronorrhina derbyana,M,2.120,6,0.039532,0.473369
4,Cetoniinae,Dicronorrhina derbyana,F,2.145,6,0.049701,0.496320


For some of this analysis, we will look at the per-species averages for our measurements. To get this, we use a simple aggregate function on the dataframe and take a look at the results.

In [5]:
df_averages = df.groupby(['species', 'spiracle'], as_index=False).aggregate(np.average)
df_averages.head()

,species,spiracle,area (mm^2),depth (mm),mass (g)
0,Coelorrhina hornimani,1,0.135347,0.416717,1.13
1,Coelorrhina hornimani,2,0.084207,0.451409,1.13
2,Coelorrhina hornimani,3,0.106693,0.325444,1.13
3,Coelorrhina hornimani,4,0.115574,0.481558,1.13
4,Coelorrhina hornimani,5,0.119145,0.506751,1.13


For our plots, we will log transform the data. We will add a column to the dataframe with the log transformed data.

In [6]:
df_averages['log area (mm^2)'] = np.log10(df_averages['area (mm^2)'])
df_averages['log dist'] = np.log10(df_averages['depth (mm)'])
df_averages['log mass (g)'] = np.log10(df_averages['mass (g)'])
df_averages['log area/dist'] = np.log10(df_averages['area (mm^2)']/df_averages['depth (mm)'])
df_averages['log area^2/dist'] = np.log10(df_averages['area (mm^2)']**2/df_averages['depth (mm)'])

df_averages.head()

,species,spiracle,area (mm^2),depth (mm),mass (g),log area (mm^2),log dist,log mass (g),log area/dist,log area^2/dist
0,Coelorrhina hornimani,1,0.135347,0.416717,1.13,-0.868551,-0.380159,0.053078,-0.488392,-1.356943
1,Coelorrhina hornimani,2,0.084207,0.451409,1.13,-1.074651,-0.345430,0.053078,-0.729221,-1.803872
2,Coelorrhina hornimani,3,0.106693,0.325444,1.13,-0.971862,-0.487524,0.053078,-0.484339,-1.456201
3,Coelorrhina hornimani,4,0.115574,0.481558,1.13,-0.937142,-0.317351,0.053078,-0.619790,-1.556932
4,Coelorrhina hornimani,5,0.119145,0.506751,1.13,-0.923923,-0.295205,0.053078,-0.628717,-1.552640


In addition to log transforming the species averaged data, we will do the same for the whole data set.

In [7]:
df['log area (mm^2)'] = np.log10(df['area (mm^2)'])
df['log dist'] = np.log10(df['depth (mm)'])
df['log mass (g)'] = np.log10(df['mass (g)'])
df['log area/dist'] = np.log10(df['area (mm^2)']/df['depth (mm)'])
df['log area^2/dist'] = np.log10(df['area (mm^2)']**2/df['depth (mm)'])
df.head()

,subfamily,species,sex,mass (g),spiracle,area (mm^2),depth (mm),log area (mm^2),log dist,log mass (g),log area/dist,log area^2/dist
0,Cetoniinae,Goliathus goliathus,M,16.280,6,0.274408,2.512648,-0.561603,0.400132,1.211654,-0.961735,-1.523338
1,Cetoniinae,Goliathus goliathus,F,18.150,6,0.134949,1.606189,-0.869831,0.205797,1.258877,-1.075628,-1.945459
2,Cetoniinae,Coelorrhina hornimani,M,1.130,6,0.212131,0.553833,-0.673395,-0.256621,0.053078,-0.416774,-1.090169
3,Cetoniinae,Dicronorrhina derbyana,M,2.120,6,0.039532,0.473369,-1.403054,-0.324800,0.326336,-1.078254,-2.481309
4,Cetoniinae,Dicronorrhina derbyana,F,2.145,6,0.049701,0.496320,-1.303635,-0.304238,0.331427,-0.999397,-2.303033


Now we can start to generate some plots for the data and see what we are dealing with. I first define a couple of functions to plot lines representing isometric scaling. 

In [8]:
def generate_line(slope, intercept, point=0, move=100):
    x1 = point-move
    x2 = point+move
    y1 = slope*x1 + intercept
    y2 = slope*x2 + intercept
    return (x1, x2), (y1, y2)

def first_intercept(slope, x_max, y_min):
    return(y_min-slope*x_max)

In order to get confidence intervals for the regressions, we need a function to do bootstrap replicates. To do this, you simply draw samples (with replacement) from the data. With this sample, you then perform the regression again. Doing this over and over again gives boostrap samples from which confidence intervals can be computed. 

In [9]:
def draw_bs_pairs_linreg(x, y, size=1):
    """Perform pairs bootstrap for linear regression."""
    # Set up array of indices to sample from
    inds = np.arange(len(x))

    # Initialize samples
    bs_slope_reps = np.empty(size)
    bs_intercept_reps = np.empty(size)

    # Take samples
    for i in range(size):
        bs_inds = np.random.choice(inds, len(inds))
        bs_x, bs_y = x[bs_inds], y[bs_inds]
        bs_slope_reps[i], bs_intercept_reps[i] = np.polyfit(bs_x, bs_y, deg=1)

    return bs_slope_reps, bs_intercept_reps

Here is a function that performs the bootstrap sampling and builds a bokeh plot for the data. 

In [12]:
def make_plot(df, to_plot, slope_comp, n_cols=4):
    
    plots = []

    for spiracle in ['S', 'T', '1', '2', '3', '4', '5', '6']:

        y_min, y_max = np.min(df[to_plot].values), np.max(df[to_plot].values)
        x_min, x_max = np.min(df['log mass (g)'].values), np.max(df['log mass (g)'].values)
        intercept1 = first_intercept(slope_comp, x_max, y_min)
        line_scale = (y_max - y_min)/10

        p = bokeh.plotting.figure(width=230, height=230,
                                  y_range=(y_min-0.2, y_max+0.2),
                                  x_range=(x_min-0.2, x_max+0.2)
                                 )
        [p.line(generate_line(intercept=i, slope=slope_comp, point=x_max)[0],
                generate_line(intercept=i, slope=slope_comp, point=x_max)[1], color='grey', alpha=0.3)
         for i in line_scale*np.array(range(30))+intercept1]

        p.scatter('log mass (g)', to_plot,
                  source = df.loc[(df['spiracle'] == spiracle)])

        #p.legend.location = 'bottom_right'
        p.xgrid.visible = False
        p.ygrid.visible = False
        
        slope, intercept = np.polyfit(df_averages.loc[(df_averages['spiracle'] == spiracle), 'log mass (g)'].values, 
                              df_averages.loc[(df_averages['spiracle'] == spiracle), to_plot].values, deg=1)
        x = np.array([x_min, x_max])
        y = slope * x + intercept

        p.line(x, y)
        
        bs_slope_reps, bs_intercept_reps = draw_bs_pairs_linreg(
                        df.loc[(df['spiracle'] == spiracle), 'log mass (g)'].values,
                        df.loc[(df['spiracle'] == spiracle), to_plot].values,
                                                        size=10000)
        
        p.title.text = spiracle + ' slope 95% CI: ' + str([round(j, 3) for j in np.percentile(bs_slope_reps, [2.5, 97.5])])

        # x-values
        x = np.linspace(x_min, x_max, 200)

        # y-values of each point
        y = np.outer(bs_slope_reps, x) + np.stack([bs_intercept_reps]*200, axis=1)

        # Compute the 2.5th and 97.5th percentiles
        low, high = np.percentile(y, [2.5, 97.5], axis=0)
        
        p1 = np.append(x, x[::-1])
        p2 = np.append(low, high[::-1])

        p.patch(p1, p2, alpha=0.5)

        plots.append(p)
        

    bokeh.io.show(bokeh.layouts.gridplot(plots,ncols=n_cols))

---

## Plot for species averaged mass vs species averaged spiracle area (log transformed)

In [13]:
make_plot(df_averages, 'log area (mm^2)', 2/3)

---

## Species averaged mass vs species averaged spiracle depth (log transformed)

In [14]:
make_plot(df_averages, 'log dist', 1/3)

---

## Species averaged mass vs $\frac{\mathrm{species\,averaged\,area}}{\mathrm{species\,averaged\,depth}}$ (log transformed)

In [15]:
make_plot(df_averages, 'log area/dist', 1/3)

---

## Species averaged mass vs $\frac{(\mathrm{species\,averaged\,area})^2}{\mathrm{species\,averaged\,depth}}$ (log transformed)

In [16]:
make_plot(df_averages, 'log area^2/dist', 1)

---
## All data points (not species averaged) for mass vs area (log transformed)

In [17]:
make_plot(df, 'log area (mm^2)', 2/3)

---

## All data points (not species averaged) for mass vs spiracle depth (log transformed)

In [18]:
make_plot(df, 'log dist', 1/3)

---

## All data points (not species averaged) for mass vs $\frac{\mathrm{area}}{\mathrm{depth}}$ (log transformed)

In [19]:
make_plot(df, 'log area/dist', 1/3)

---

## All data points (not species averaged) for mass vs $\frac{(\mathrm{area})^2}{\mathrm{depth}}$ (log transformed)

In [20]:
make_plot(df, 'log area^2/dist', 1)